Data set available at [this link](https://economistsview.typepad.com/economics421/files/Data-for-Problem-3.xls).

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('../Lecture04/Data-for-Problem-3.xls', usecols=[1,2])

In [3]:
data.head()

,SALARY,YEARS
0,105.2,36
1,91.3,30
2,72.5,29
3,74.3,28
4,103.5,26


Linear model:

$$
\ln\ \textrm{Salary} = \beta_1 + \beta_2\ \textrm{years} + \beta_3\ \textrm{years}^2 + u_i, \quad \textrm{Var}(u_i) = \sigma^2_i
$$

In [4]:
data['LOG_SALARY'] = data.SALARY.apply(np.log)

In [5]:
data['YEARS_SQ'] = data.YEARS.pow(2)

In [6]:
data.head()

,SALARY,YEARS,LOG_SALARY,YEARS_SQ
0,105.2,36,4.655863,1296
1,91.3,30,4.514151,900
2,72.5,29,4.283587,841
3,74.3,28,4.308111,784
4,103.5,26,4.639572,676


Regressing $\ln\ \textrm{Salary}$

In [7]:
from sklearn.linear_model import LinearRegression

In [8]:
reg = LinearRegression()

In [9]:
X = data[['YEARS', 'YEARS_SQ']]
y = data['LOG_SALARY']

In [10]:
reg.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [11]:
print(reg.intercept_, reg.coef_)

3.8093653545744024 [ 0.04385284 -0.00062735]


Variance model:
$$
\sigma^2_i = \alpha_1 + \alpha_2\ \textrm{years} + \alpha_3\ \textrm{years}^2 + \alpha_4\ \big(\textrm{years}^2\big)^2 + \alpha_5\ \textrm{years} \cdot \textrm{years}^2
$$

In [12]:
y_hat = reg.predict(X)

In [13]:
u_hat = y - y_hat

In [14]:
u_hat_sq = u_hat.pow(2)

Regressing $\sigma^2_i$ using $\hat u^2_i$

In [15]:
var_reg = LinearRegression()

In [16]:
X_var = pd.concat([X, data['YEARS_SQ'].pow(2), data['YEARS'].multiply(data['YEARS_SQ'])], axis=1)
X_var.columns = ['YEARS', 'YEARS_SQ', 'YEARS_SQ_SQ', 'YEARS_YEARS_SQ']

In [17]:
y_var = u_hat_sq

In [18]:
var_reg.fit(X_var, y_var)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [19]:
print(var_reg.intercept_, var_reg.coef_)

-0.0017701488506047108 [ 1.67112076e-04  6.46661541e-04  4.22235547e-07 -3.26366936e-05]


Performing the test:

In [20]:
r_squared = var_reg.score(X_var, y_var)

In [21]:
n, p_minus_one = X_var.shape # the constant term was not added to the X matrix!

In [22]:
white_stat = n * r_squared

In [23]:
from scipy.stats import chi2

In [24]:
alpha = 0.05
df = p_minus_one

In [25]:
chi2_crit = chi2.ppf(q=1-alpha,df=df)

In [26]:
if white_stat > chi2_crit:
    print('REJECT NULL HYPOTHESIS! -> There is heteroskedasticity.')
else:
    print('Failed to reject the hull hypothesis.')

REJECT NULL HYPOTHESIS! -> There is heteroskedasticity.
